In [1]:
from data.bloomberg_api import BlpQuery

In [2]:
bquery = BlpQuery().start()

In [37]:
INDEXES = bquery.bdh(
    ["RTY Index", "SPX Index", "SX5E Index", "SXXP Index"],
    ["PX_LAST"],
    start_date="20000101",
    end_date="20240419",
    options={"adjustmentSplit": True},
)

In [53]:
dfi = INDEXES.pivot(index="date",columns="security",values="PX_LAST").fillna(method="ffill").asfreq('1B',method="ffill")#.to_excel("../data/Equity_index.xlsx")

In [54]:
dfi.head(20)

security,RTY Index,SPX Index,SX5E Index,SXXP Index
date,,,,
2000-01-03,498.122,1455.22,4849.22,377.69
2000-01-04,479.035,1399.42,4657.83,362.70
2000-01-05,479.457,1402.11,4541.75,353.74
2000-01-06,475.892,1403.45,4500.69,352.21
2000-01-07,489.011,1441.47,4648.27,359.75
2000-01-10,502.569,1457.60,4714.03,365.39
2000-01-11,493.255,1438.56,4671.50,362.93
2000-01-12,490.613,1432.25,4649.30,362.92
2000-01-13,501.851,1449.68,4688.35,364.83


In [84]:
INDEXES.pivot(index="date",columns="security",values="PX_LAST").to_excel("../data/Equity_index.xlsx")

In [85]:
import pandas as pd
df_i = pd.read_excel("../data/Equity_index.xlsx",index_col="date").fillna(method="ffill").asfreq('1B',method="ffill")
df_i

,RTY Index,SPX Index,SX5E Index,SXXP Index
date,,,,
2000-01-03,498.122,1455.22,4849.22,377.69
2000-01-04,479.035,1399.42,4657.83,362.70
2000-01-05,479.457,1402.11,4541.75,353.74
2000-01-06,475.892,1403.45,4500.69,352.21
2000-01-07,489.011,1441.47,4648.27,359.75
...,...,...,...,...
2024-04-15,1975.708,5061.82,4984.48,505.93
2024-04-16,1967.475,5051.41,4916.99,498.21
2024-04-17,1947.947,5022.21,4914.13,498.52


In [18]:
df

,security,field,secondary_name,secondary_value,value
0,GETIB SS Equity,#Filtered_Data,ACTION_ID,155331421,0.821721
1,CA FP Equity,#Filtered_Data,ACTION_ID,52444053,0.873578
2,AC FP Equity,#Filtered_Data,ACTION_ID,43512962,0.649832
3,VIV FP Equity,#Filtered_Data,ACTION_ID,226128288,0.290730
4,ELUXB SS Equity,#Filtered_Data,ACTION_ID,186613054,0.824895
...,...,...,...,...,...
190,ATCOA SS Equity,#Filtered_Data,TYPE,Spin off,0.772147
191,SOP FP Equity,#Filtered_Data,TYPE,Spin off,0.702517
192,NKT DC Equity,#Filtered_Data,TYPE,Spin off,0.489308
193,RKT LN Equity,#Filtered_Data,TYPE,Spin off,0.977401


In [35]:
from typing import Literal, Union
from datetime import date, datetime
import pandas as pd


def get_spin_off_history(
    index_universe: Literal["RTY Index", "SPX Index", "SX5E Index", "SXXP Index"],
    start_date: Union[date, datetime, str],
    end_date: Union[date, datetime, str],
) -> pd.DataFrame:
    """_summary_

    Args:
        index_universe (Literal[&quot;RTY Index&quot;, &quot;SPX Index&quot;, &quot;SX5E Index&quot;, &quot;SXXP Index&quot;]): _description_
        start_date (Union[date, datetime, str]): _description_
        end_date (Union[date, datetime, str]): _description_

    Returns:
        pd.DataFrame: Returns a pd.DataFrame with columns : SPINOFF_TICKER_PARENT	ANNOUNCED_DATE	EFFECTIVE_DATE	SPINOFF_TICKER
    """    
    assert index_universe in {
        "RTY Index",
        "SPX Index",
        "SX5E Index",
        "SXXP Index",
    }, "Error, provide a valid index universe."
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    bquery = BlpQuery().start()
    spin_off_raw_dataframe = bquery.bql(
        f"""let(#Data = Spinoffs(Effective_Date=range({start_date.strftime('%Y-%m-%d')},{end_date.strftime('%Y-%m-%d')}));
                #Filtered_Data = dropna(matches(#Data,#Data().DISTRIBUTION_RATIO >= 0.0),true);)
                get(#Filtered_Data)
                for(members('{index_universe}'))
                with(currency=USD)
                preferences(addcols=all)"""
    )
    bquery.stop()
    return (
        (
            spin_off_raw_dataframe.pivot_table(
                values=["secondary_value"],
                columns="secondary_name",
                index="security",
                aggfunc="first",
            )["secondary_value"]
        )
        .reset_index()[
            ["security", "ANNOUNCED_DATE", "EFFECTIVE_DATE", "SPINOFF_TICKER"]
        ]
        .rename(columns={"security": "SPINOFF_TICKER_PARENT"})
    )

In [36]:
dffff = get_spin_off_history('SPX Index',start_date="2020-01-01",end_date="2024-01-01")
dffff

secondary_name,SPINOFF_TICKER_PARENT,ANNOUNCED_DATE,EFFECTIVE_DATE,SPINOFF_TICKER
0,BBWI UN Equity,2021-05-11T00:00:00Z,2021-08-03T00:00:00Z,VSCO US Equity
1,BDX UN Equity,2021-05-06T00:00:00Z,2022-04-01T00:00:00Z,EMBC US Equity
2,BWA UN Equity,2023-02-09T00:00:00Z,2023-07-05T00:00:00Z,PHIN US Equity
3,DD UN Equity,2021-01-07T00:00:00Z,2022-12-31T00:00:00Z,1784122D US Equity
4,DHR UN Equity,2022-09-14T00:00:00Z,2023-10-02T00:00:00Z,VLTO US Equity
5,DTE UN Equity,2020-10-27T00:00:00Z,2021-07-01T00:00:00Z,DTM US Equity
6,ECL UN Equity,2019-02-04T00:00:00Z,2020-06-04T00:00:00Z,1695496D US Equity
7,ETN UN Equity,2019-03-01T00:00:00Z,2021-12-31T00:00:00Z,1700735D US Equity
8,EXC UW Equity,2021-03-03T00:00:00Z,2022-02-02T00:00:00Z,CEG US Equity
9,FTV UN Equity,2019-09-04T00:00:00Z,2021-12-31T00:00:00Z,1745856D US Equity


In [34]:
(
    df.pivot_table(
        values=["secondary_value"],
        columns="secondary_name",
        index="security",
        aggfunc="first",
    )["secondary_value"]
).reset_index()[
    ["security", "ANNOUNCED_DATE", "EFFECTIVE_DATE", "SPINOFF_TICKER"]
].rename(
    columns={"security": "SPINOFF_TICKER_PARENT"}
)

secondary_name,SPINOFF_TICKER_PARENT,ANNOUNCED_DATE,EFFECTIVE_DATE,SPINOFF_TICKER
0,AC FP Equity,2010-03-02T00:00:00Z,2010-07-02T00:00:00Z,EDEN FP Equity
1,ATCOA SS Equity,2018-01-26T00:00:00Z,2018-06-13T00:00:00Z,EPIA SS Equity
2,BKT SQ Equity,2021-03-23T00:00:00Z,2021-04-29T00:00:00Z,LDA SM Equity
3,CA FP Equity,2011-03-01T00:00:00Z,2011-07-05T00:00:00Z,DIA SM Equity
4,ELUXB SS Equity,2019-02-04T00:00:00Z,2020-03-18T00:00:00Z,EPROB SS Equity
5,GETIB SS Equity,2017-11-10T00:00:00Z,2017-12-07T00:00:00Z,ARJOB SS Equity
6,NKT DC Equity,2017-09-11T00:00:00Z,2017-10-12T00:00:00Z,NLFSK DC Equity
7,PRU LN Equity,2018-03-14T00:00:00Z,2019-10-21T00:00:00Z,MNG LN Equity
8,RKT LN Equity,2014-07-28T00:00:00Z,2014-12-23T00:00:00Z,INDV LN Equity
9,SCAB SS Equity,2016-08-30T00:00:00Z,2017-06-12T00:00:00Z,ESSITYB SS Equity


In [11]:
df.secondary_name.value_counts()

ACTION_ID             13
ANNOUNCED_DATE        13
CURRENCY              13
DATE                  13
DISTRIBUTION_RATIO    13
EFFECTIVE_DATE        13
EXISTING_SHARES       13
FRACTIONAL_SHARES     13
NEW_SHARES            13
PAY_DATE              13
RECORD_DATE           13
SPINOFF_TICKER        13
STATUS                13
TRANSACTION_VALUE     13
TYPE                  13
Name: secondary_name, dtype: int64